# Feature Engineering

## 1. Introducción al Feature Engineering

### 1.1 Objetivo del Feature Engineering
Pendiente: describir el objetivo de preparar y transformar las variables para su uso en modelos de Machine Learning.

### 1.2 Contexto del proceso de preparación de datos
Pendiente: explicar el papel del Feature Engineering dentro del flujo general del análisis.

---

## 2. Selección inicial de variables

### 2.1 Identificación de variables candidatas
Pendiente: listar variables macroeconómicas y fiscales seleccionadas para el modelado.

### 2.2 Exclusión de variables no relevantes
Pendiente: justificar la exclusión de identificadores u otras variables no predictivas.

---

## 3. Tratamiento de valores faltantes

### 3.1 Detección de valores faltantes
Pendiente: identificar variables con valores nulos o faltantes.

### 3.2 Estrategias de imputación
Pendiente: definir métodos de imputación aplicables según el tipo de variable.

### 3.3 Verificación post-imputación
Pendiente: comprobar que no quedan valores faltantes tras el tratamiento.

---

## 4. Transformación de variables

### 4.1 Escalado y normalización
Pendiente: aplicar técnicas de escalado cuando sea necesario.

### 4.2 Transformaciones funcionales
Pendiente: aplicar logaritmos u otras transformaciones si la distribución lo requiere.

---

## 5. Creación de nuevas variables

### 5.1 Variables rezagadas
Pendiente: creación de variables basadas en información del período anterior.

### 5.2 Ratios e indicadores derivados
Pendiente: construcción de ratios fiscales y macroeconómicos relevantes.

---

## 6. Análisis de multicolinealidad

### 6.1 Detección de correlaciones elevadas
Pendiente: análisis de correlaciones entre variables explicativas.

### 6.2 Decisiones de eliminación o combinación
Pendiente: justificar la eliminación o combinación de variables altamente correlacionadas.

---

## 7. Selección final de variables

### 7.1 Conjunto final de variables explicativas
Pendiente: definición del conjunto final de variables para el modelado.

### 7.2 Comparación de subconjuntos de variables
Pendiente: evaluar diferentes combinaciones de variables si aplica.

---

## 8. Dataset final para modelado

### 8.1 Construcción del dataset final
Pendiente: ensamblar el dataset definitivo con variables explicativas y variable objetivo.

### 8.2 Verificación de consistencia
Pendiente: comprobar dimensiones, tipos y ausencia de valores faltantes.

---

## 9. Cierre del Feature Engineering

### 9.1 Resumen del proceso
Pendiente: resumen de las transformaciones aplicadas.

### 9.2 Preparación del dataset final
Pendiente: dejar el dataset listo para el entrenamiento de modelos.


# Notebook 02 — Feature Engineering

Este notebook está dedicado a la construcción y preparación de las variables explicativas que serán utilizadas en los modelos de Machine Learning supervisado.


## Construcción de variables explicativas

Las variables explicativas se construyen utilizando información del período anterior (t−1), de acuerdo con el planteamiento metodológico del proyecto.

**Pendiente:** generar las variables explicativas con retardo temporal.


## Tratamiento de valores faltantes

Se analizará la presencia de valores faltantes y se aplicarán estrategias de imputación cuando sea necesario.

**Pendiente:** definir y aplicar el método de imputación adecuado.


## Normalización y escalado

Se aplicarán técnicas de normalización o escalado a las variables explicativas para facilitar el entrenamiento de los modelos.

**Pendiente:** aplicar escalado a las variables seleccionadas.
